In [1]:
from arcgis.gis import GIS
import pandas as pd

api_key = "AAPKcbf8118c14454e038f0f8f49acd67ecbK881HtN8H-5A3V1Rydhsy781gAIvcTSuytUP7ctrJS76H_MIqra_ii2zLyhUfngq"
gis = GIS(api_key=api_key)

housing_layer = gis.content.get("bf0e7c80db674a85ac58d63a7fc6939f").layers[0]

housing_df = pd.DataFrame.spatial.from_layer(housing_layer)
housing_df.head(1)
# housing_layer.properties

,OBJECTID_1,OBJECTID,ADDRESS,OWNERSHIP_,BUILDING_U,USE_CODE,USE_DESCRI,USE_LONG_D,AGENCY,OWNERNAME,...,SHAPELEN,ORIG_FID,Image,Availability,GlobalID,CreationDate,Creator,EditDate,Editor,SHAPE
0,1,26660,3269 STANTON RD SE WASHINGTON DC 20020,PUBLIC,PUBLIC HOUSING,011,Residential-Row-Single-Family,(Class 1 or 2) : Single family dwelling with 2...,DCHA,DISTRICT OF COLUMBIA HOUSING AUTHORITY,...,0,1,<NA>,Available,ba112ee8-df04-4310-8c57-d823aa61cda5,2023-07-22 13:11:58.130000114,hlarsen_intern_hackathon,2023-07-22 13:11:58.130000114,hlarsen_intern_hackathon,"{""x"": -76.980770955715, ""y"": 38.84726619387982..."


In [241]:
query_results = housing_layer.query(where="OBJECTID=26660")
query_results.features[0]

{"geometry": {"x": -76.980770955715, "y": 38.84726619387982, "spatialReference": {"wkid": 4326, "latestWkid": 4326}}, "attributes": {"OBJECTID_1": 1, "OBJECTID": 26660, "ADDRESS": "3269 STANTON RD SE WASHINGTON DC 20020", "OWNERSHIP_": "PUBLIC", "BUILDING_U": "PUBLIC HOUSING", "USE_CODE": "011", "USE_DESCRI": "Residential-Row-Single-Family", "USE_LONG_D": "(Class 1 or 2) : Single family dwelling with 2 walls build as common walls with another structure, 2 exposed walls; primarily used as a place of abode.", "AGENCY": "DCHA", "OWNERNAME": "DISTRICT OF COLUMBIA HOUSING AUTHORITY", "OWNERNAME2": " ", "WARD": 8, "ANC": "ANC 8E", "ZONING": "R-3", "SITENAME": "HENSON RIDGE:  3269 STANTON ROAD SE", "PROPTYPE": "Residential-Single Family (Row", "UNITS": 0, "MAR_ID": 155584, "SHAPEAREA": 0, "SHAPELEN": 0, "ORIG_FID": 1, "Image": null, "Availability": "Available", "GlobalID": "ba112ee8-df04-4310-8c57-d823aa61cda5", "CreationDate": 1690031518130, "Creator": "hlarsen_intern_hackathon", "EditDate":

In [9]:
type(housing_layer)

arcgis.features.layer.FeatureLayer

In [21]:
import arcpy

arcpy.FeatureClassToFeatureClass_conversion("C:\\Users\\gre13341\\Documents\\Hackathon\\New_File_Geodatabase.gdb\\PublicHousing_Points", "C:\\Users\\gre13341\\Documents\\Hackathon", "single_house")

<Result 'C:\\Users\\gre13341\\Documents\\Hackathon\\single_house.shp'>

In [23]:
import arcpy

single_house = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=r"C:\Users\gre13341\Documents\Hackathon\New_File_Geodatabase.gdb\PublicHousing_Points",
    selection_type="NEW_SELECTION",
    where_clause="OBJECTID = 26689",
    invert_where_clause=None
)
type(single_house)

arcpy.arcobjects.arcobjects.Result

In [25]:
single_house = arcpy.analysis.Select(
    r"C:\Users\gre13341\Documents\Hackathon\New_File_Geodatabase.gdb\PublicHousing_Points", 
    r"C:\Users\gre13341\Documents\Hackathon\New_File_Geodatabase.gdb\single_house", 
    where_clause="OBJECTID = 26689"
)
type(single_house)

ExecuteError: Failed to execute. Parameters are not valid.
ERROR 000725: Output Feature Class: Dataset C:\Users\gre13341\Documents\Hackathon\New_File_Geodatabase.gdb\single_house already exists.
Failed to execute (Select).


In [167]:
facilities = f"{housing_df.SHAPE[0]['x']},{housing_df.SHAPE[0]['y']}"
facilities

'-76.980770955715,38.84726619387982'

In [207]:
import arcgis

def print_result(result):
    """Print useful information from the result."""
    pd.set_option("display.max_rows", None)
    pd.set_option("display.max_colwidth", None)

    output_polygons = arcgis.features.FeatureSet.from_dict(result["saPolygons"]).sdf
    print("\n-- Output Polygons -- \n")
    print(output_polygons[["Name", "FromBreak", "ToBreak"]].to_string(index=False))

service_area = arcgis.network.ServiceAreaLayer(gis.properties.helperServices.serviceArea.url, gis=gis)

# Get the walking distance travel mode defined for the portal. Fail if the travel mode is not found.
walking_distance_travel_mode = ""
for feature in arcgis.network.analysis.get_travel_modes().supported_travel_modes.features:
    attributes = feature.attributes
    if attributes["AltName"] == "Walking Time":
        walking_distance_travel_mode = attributes["TravelMode"]
        break
assert walking_distance_travel_mode, "Walking Distance travel mode not found"

result = service_area.solve_service_area(facilities=facilities,
                                            default_breaks=[20],
                                            travel_direction="esriNATravelDirectionToFacility",
                                            travel_mode=walking_distance_travel_mode,
                                            out_sr=3857)
print_result(result)

Cannot find {"attributeParameterValues": [{"attributeName": "Avoid Private Roads", "parameterName": "Restriction Usage", "value": "AVOID_MEDIUM"}, {"attributeName": "Walking", "parameterName": "Restriction Usage", "value": "PROHIBITED"}, {"attributeName": "Preferred for Pedestrians", "parameterName": "Restriction Usage", "value": "PREFER_LOW"}, {"attributeName": "WalkTime", "parameterName": "Walking Speed (km/h)", "value": 5}, {"attributeName": "Avoid Roads Unsuitable for Pedestrians", "parameterName": "Restriction Usage", "value": "AVOID_HIGH"}], "description": "Follows paths and roads that allow pedestrian traffic and finds solutions that optimize travel time. The walking speed is set to 5 kilometers per hour.", "distanceAttributeName": "Kilometers", "id": "caFAgoThrvUpkFBW", "impedanceAttributeName": "WalkTime", "name": "Walking Time", "restrictionAttributeNames": ["Avoid Private Roads", "Avoid Roads Unsuitable for Pedestrians", "Preferred for Pedestrians", "Walking"], "simplificati

AttributeError: 'ServiceAreaLayer' object has no attribute 'solve'

In [177]:
polygon_dict = result['saPolygons']['features'][0]['geometry']
polygon_dict

{'rings': [[[-8561585.8121, 4714072.0594],
   [-8561485.8126, 4714007.770000003],
   [-8561460.8127, 4713782.760799997],
   [-8561610.812, 4713364.899599999],
   [-8561835.811, 4713075.621200003],
   [-8561735.8114, 4713011.338200003],
   [-8561685.8116, 4712882.773500003],
   [-8561785.8111, 4712754.2104],
   [-8562035.81, 4712786.3510000035],
   [-8562160.8094, 4712432.809799999],
   [-8561610.812, 4712336.391599998],
   [-8561510.8125, 4712207.835500002],
   [-8561535.8123, 4712111.419299997],
   [-8561610.812, 4712047.1426],
   [-8562010.8102, 4712111.419299997],
   [-8562010.8102, 4712047.1426],
   [-8562110.8097, 4711982.865999997],
   [-8562260.809, 4712143.557999998],
   [-8562285.8089, 4712079.2808],
   [-8562460.8081, 4712015.004299998],
   [-8562560.8076, 4711629.352799997],
   [-8562635.8072, 4711565.079000004],
   [-8562685.8071, 4711597.215899996],
   [-8562785.8066, 4711436.532700002],
   [-8562960.8058, 4711372.259999998],
   [-8562860.8063, 4711179.444700003],
   [-856

In [205]:
polygon = arcgis.geometry.Polygon(polygon_dict)
type(polygon)

arcgis.geometry._types.Polygon

In [192]:
from arcgis.features import FeatureLayer

metro_layer = FeatureLayer("https://maps2.dcgis.dc.gov/dcgis/rest/services/DCGIS_DATA/Transportation_Rail_Bus_WebMercator/MapServer/52")
metro_df = pd.DataFrame.spatial.from_layer(metro_layer)
takoma_pt = arcgis.geometry.Point(metro_df.SHAPE[0])

In [199]:
ch = metro_layer.query(where="NAME='Congress Heights'")
type(ch)

arcgis.features.feature.FeatureSet

In [197]:
congress_heights_pt = arcgis.geometry.Point(metro_df.loc[39]['SHAPE'])
congress_heights_pt = arcgis.geometry.project(geometries=[congress_heights_pt], in_sr=3857, out_sr=4326)[0]
str(congress_heights_pt)


"{'x': -76.98851420349799, 'y': 38.84566547961086, 'spatialReference': {'wkid': 4326, 'latestWkid': 4326}}"

In [174]:
polygon.overlaps(congress_heights_pt)

False

In [59]:
import arcpy

with open("password.txt", 'r') as pw_file:
    pw = pw_file.read()

arcpy.SignInToPortal("https://www.arcgis.com/", "gpaiement_intern_hackathon", pw)

# Instantiate a ServiceArea analysis object.
service_area = arcpy.nax.ServiceArea("https://www.arcgis.com/")
service_area

In [60]:
# Get the desired travel mode for the analysis.
nd_layer_name = "https://www.arcgis.com/"
nd_travel_modes = arcpy.nax.GetTravelModes(nd_layer_name)
travel_mode = nd_travel_modes["Walking Time"]

# Set properties.
service_area.timeUnits = arcpy.nax.TimeUnits.Minutes
service_area.defaultImpedanceCutoffs = [20]
service_area.travelMode = travel_mode
service_area.outputType = arcpy.nax.ServiceAreaOutputType.Polygons
service_area.geometryAtOverlap = arcpy.nax.ServiceAreaOverlapGeometry.Split


In [61]:
service_area.load(arcpy.nax.ServiceAreaInputDataType.Facilities, r"C:\Users\gre13341\Documents\Hackathon\New_File_Geodatabase.gdb\single_house")

In [62]:
result = service_area.solve()

In [63]:
result.solveSucceeded

True

In [237]:
# Print all the warning messages.
for message in result.solverMessages(arcpy.nax.MessageSeverity.Warning):
    print(message[-1])
# Print all the error messages.
for message in result.solverMessages(arcpy.nax.MessageSeverity.Error):
    print(message[-1])

ERROR 030117: Insufficient input. Need at least one facility and one break value.
Failed to execute (GenerateServiceAreas).
Failed.


In [65]:
output_type = arcpy.nax.ServiceAreaOutputDataType.Polygons
result.export(output_type, r"C:\Users\gre13341\Documents\Hackathon\New_File_Geodatabase.gdb\polygon")

In [70]:
test = arcpy.management.SelectLayerByLocation(
    in_layer=r"C:\Users\gre13341\Documents\Hackathon\New_File_Geodatabase.gdb\Metro_Stations",
    overlap_type="INTERSECT",
    select_features=r"C:\Users\gre13341\Documents\Hackathon\New_File_Geodatabase.gdb\polygon",
    search_distance=None,
    selection_type="NEW_SELECTION",
    invert_spatial_relationship="NOT_INVERT"
)

In [71]:
arcpy.management.CopyFeatures(test, r"C:\Users\gre13341\Documents\Hackathon\New_File_Geodatabase.gdb\final_result2")

<Result 'C:\\Users\\gre13341\\Documents\\Hackathon\\New_File_Geodatabase.gdb\\final_result2'>